In [ ]:
import numpy as np
import pandas as pd
from nltk.util import pairwise

import gurobipy as gp
from gurobipy import GRB

import os
import ipdb
import ipynb
from pathlib import Path
from csv import DictWriter

from importlib import reload
import ipynb.fs.full.A5_simulation as simulation

# Scheduling via Gurobi

In [ ]:
def get_indices(roomAssignment):
    
    ir_ = []
    ijr_ = []
    
    for room in roomAssignment.room.unique():
        
        df = roomAssignment[roomAssignment.room == room]
        df = df[df.scenario == 0]
        
        ir_.append(tuple((i,room) for i in df.index))
        
        if df.shape[0] > 1:
        
            ijr_.append(tuple((i,j,room) for i, j in zip(df.index, df.index[1:])))
    
    ir_ = [z for y in (x if isinstance(x[0],tuple) else [x] for x in ir_) for z in y]
    ijr_ = [z for y in (x if isinstance(x[0],tuple) else [x] for x in ijr_) for z in y]
    
    return ir_, ijr_

In [ ]:
def randomRoomAssignment(treatmentData, roomData, scenarioData):
    
    roomAssignment = treatmentData.copy()
    roomAssignment['room'] = 6 # add dummy room
    roomAssignment = roomAssignment[roomAssignment.index < 11111] # remove dummy patients
    counter = roomAssignment.shape[0]
    
    while counter > 0:
        
        for room in roomData.index:

            # get pool of yet available and fitting treatments for room
            pool = roomAssignment[(roomAssignment.type.isin(roomData.type[room])) & (roomAssignment.room > 5)]

            # check if pool is empty
            if not pool.empty:
                
                # set seed
                np.random.seed(42)
                
                # randomly select one patient from pool
                random_patient = np.random.choice(pool.index, size= 1)[0]

                # assign room to that randomly selected patient
                roomAssignment.xs(random_patient)['room'] = room
                
                # remove patient from counter
                counter -= 1
                
            else:
                continue
    
    
    scenarioData = scenarioData.reset_index()
    roomAssignment = pd.merge(scenarioData, roomAssignment, on='patient_id')
    roomAssignment = roomAssignment.set_index(['patient_id'])
    
    return roomAssignment

In [ ]:
def getDeanSchedule(treatmentData, scenarioData, roomData, overTimePenalty, workingTime, timeLimit, modelOutputFlag, inputSettingDict, dataTest):
    
    if not 'punctuality' in scenarioData:
        scenarioData['punctuality'] = 0
    
    with gp.Env(empty = False) as env:
        env.start()
        with gp.Model('Sequencing', env=env) as model:
            
            ##### INITIALIZE MODEL #####
            
            model.ModelSense = GRB.MINIMIZE
            
            
            ##### SCENARIOS #####
            
            scenarios = list(set(scenarioData.index.get_level_values(level=1)))
            nScenarios = len(np.unique([index[1] for index in scenarioData.index]))
            
            
            ##### ROOM ASSIGNMENT #####
            
            roomAssignment = randomRoomAssignment(treatmentData, roomData, scenarioData)
            
            
            ##### INDICES #####
            
            ir_, ijr_ = get_indices(roomAssignment)
            
            
            ##### VARIABLES #####
            
            s_fix = model.addVars([(i,r) for i,r in ir_], vtype = GRB.CONTINUOUS, name = 'start_fix', lb = 0)
            s_var = model.addVars([(i, s, r) for i,r in ir_ for s in scenarios], vtype = GRB.CONTINUOUS, name = 'start_var', lb = 0)
            e_var = model.addVars([(i, s, r) for i,r in ir_ for s in scenarios], vtype = GRB.CONTINUOUS, name = 'end_var', lb = 0)
            e_total = model.addVars([(r, s) for r in roomData.index for s in scenarios], vtype = GRB.CONTINUOUS, name = 'end_total', lb = 0)
            
            ##### INITIALIZE VARIABLES FOR CALLBACK #####
            
            model._s_fix = s_fix
            model._inputSettingDict = inputSettingDict
            model._workingTime = workingTime
            model._dataTest = dataTest

            ##### CONSTRAINTS #####
            
            # Terminvergabe
            for (i,r) in ir_:
                for s in scenarios:
                    model.addConstr(s_var[i, s, r] >= s_fix[i, r] + scenarioData.punctuality.at[i, s]) # surgery may start early
                    model.addConstr(e_var[i, s, r] == s_var[i, s, r] + scenarioData.duration.at[i, s])
            
            # calculating waiting time
            for (i,j,r) in ijr_:
                for s in scenarios:
                    model.addConstr(s_var[j,s,r] - e_var[i,s,r] >= 0)
            
            # Overtime
            for (i,r) in ir_:
                for s in scenarios:
                    model.addConstr(e_total[r,s] >= e_var[i,s,r] - workingTime)
            
                
            ##### OBJECTIVE #####
            model.setObjective(gp.quicksum((s_var[i, s, r] - s_fix[i, r]) for i,r in ir_ for s in scenarios) + 
                               gp.quicksum(overTimePenalty * (e_total[r, s]) for r in roomData.index for s in scenarios))
            
            
            ##### RUN GUROBI #####

            if modelOutputFlag == True:
                print(65*'-' + '\nSolving\n' + 65*'-')
            model.setParam(GRB.Param.TimeLimit, timeLimit)
            # save log file
            model.setParam(GRB.Param.LogFile,#"LogFile", 
                           f'./Results/gurobiLogs/dean/{inputSettingDict.get("approach")}_KW{inputSettingDict.get("KW")}_day{inputSettingDict.get("day")}_nScenarios{inputSettingDict.get("nScenarios")}.log')
            model.setParam('LogToConsole', 0) # mutes console output
            
            #model.optimize(solution_cb) # outputs solution callback
            model.optimize()
            
            ##### SOLUTION #####
            
            bestSolution = pd.DataFrame([[i, r, s_fix[i, r].X] for i, r in ir_], columns = ['patient_id', 'room', 'planned'])
            bestSolution = bestSolution.sort_values(by=['room','planned'])
            print(bestSolution.head(10))
            runtime = model.Runtime
            print(f'runtime: {round(runtime,2)}')
            
            # check if file exists
            path = f'./Results/initialSolutions/dean/KW{inputSettingDict.get("KW")}_day{inputSettingDict.get("day")}.csv'
            if not Path(path).is_file():
                pd.DataFrame(bestSolution).to_csv(path, sep = ",", index = False)
            
            saveCurrentBestSolution(bestSolution, inputSettingDict, runtime)

            simulateDay(bestSolution, dataTest, workingTime, inputSettingDict, runtime)

In [ ]:
def saveCurrentBestSolution(bestSolution, inputSettingDict, runtime):
        
    # merge current best solution, input Settings and runtime
    bestSolutionDict = bestSolution.to_dict()
    bestSolutionDict.update(inputSettingDict)
    bestSolutionDict.update({'runtime': runtime})

    # check if file exists
    path = f'./Results/schedules/dean/schedules_{inputSettingDict.get("approach")}_KW{inputSettingDict.get("KW")}_day{inputSettingDict.get("day")}.csv'
    if not Path(path).is_file():
        pd.DataFrame(bestSolutionDict).to_csv(path, sep = ",", index = False) 
    else:
        pd.concat([pd.read_csv(path, sep = ","),pd.DataFrame(bestSolutionDict)]
                 ).to_csv(path, sep = ",", index = False)

In [ ]:
def simulateDay(bestSolution, dataTest, workingTime, inputSettingDict, runtime):
    
    scheduleDay = bestSolution.merge(dataTest[['patient_id', 'duration']].set_index('patient_id'), 
                                                    on = 'patient_id', how = 'left').sort_values(['room', 'planned'])

    #--- simulation ---

    # Reload simulation to get rid of local variables from prior simulations
    reload(simulation)

    resultsDf, overtimeDf = simulation.runSimulation(scheduleDay, workingTime)

    inputSettingDictDay = inputSettingDict.copy()

    # Add results to inputSetting
    inputSettingDict['WT_sum'] = resultsDf['waitingTime'].sum()
    inputSettingDict['OT_sum'] = overtimeDf['overtime'].sum()
    inputSettingDict['num_patients'] = resultsDf.shape[0]
    inputSettingDict['runtime'] = runtime

    #---

    # check if scheduling_results.csv exists
    if not Path('./Results/scheduling_results_dean.csv').is_file():

        pd.DataFrame(inputSettingDict, index = [0]).to_csv('./Results/scheduling_results_dean.csv', sep = ",", index = False) 

    else:

        # append resDict as row
        with open('./Results/scheduling_results_dean.csv', 'a') as f:

            print('not saved in test mode')
            """
            dictwriter_object = DictWriter(f, fieldnames = list(inputSettingDict.keys()))
            dictwriter_object.writerow(inputSettingDict)
            f.close()
            """